# How to use stored 

To use stored procedures with Dapper and C#, you need to:

* Create a SqlConnection object using a connection string.
* Create a <code>DynamicParameters</code> object and add the parameters for the stored procedure.
* Use the <code>Query</code> or <code>Execute</code> methods of Dapper and specify the <code>commandType</code> as <code>CommandType.StoredProcedure</code>.

Dynamic parameters with Dapper allow you to add parameters on demand without specifying their names, types, or directions. Dapper will automatically infer the parameter names and types from the template object’s properties, and set their directions to input by default.

For simple cases, you can add stored procedure input parameters individually to the <code>DynamicParameters</code> object. For instance, if you want to call the <code>uspGetEmployeeManagers</code> stored procedure with the value specified as the <code>@BusinessEntityID</code> parameter and map the results to a list of <code>Employee</code> objects, you can use the <code>Query</code>method like the following example:

In [1]:
// Connection string
#load "AppSettings.cs"

In [2]:
#r "nuget:Microsoft.Data.SqlClient"
#r "nuget:Dapper"

using Microsoft.Data.SqlClient;
using Dapper;
using System.Data;

Installed Packages Dapper, 2.1.28 Microsoft.Data.SqlClient, 5.1.4

In [3]:
public class Employee
{
    public int BusinessEntityID { get; set; }
    public int RecursionLevel { get; set; }   
    public string FirstName { get; set; }
    public string LastName { get; set; }
    public string ManagerFirstName { get; set; }
    public string ManagerLastName { get; set; }
}

using (var connection = new SqlConnection(connectionString))
{   
    string spName = "uspGetEmployeeManagers";
    DynamicParameters parameters = new DynamicParameters();
    parameters.Add("@BusinessEntityID", 12);

    // Execute the stored procedure and map the results to a list of Employee objects
    List<Employee> employees = connection.Query<Employee>(spName, parameters, commandType: CommandType.StoredProcedure).ToList();

    // Print the results
    foreach (var employee in employees)
    {
        Console.WriteLine($@"{employee.RecursionLevel}, {employee.BusinessEntityID}: {employee.FirstName} {employee.LastName} - {employee.ManagerFirstName} {employee.ManagerLastName}");
    }
}

0, 12: Thierry D'Hers - Ovidiu Cracium
1, 11: Ovidiu Cracium - Roberto Tamburello
2, 3: Roberto Tamburello - Terri Duffy


When the stored procedure has multiple input parameters, you can wrap all of them in a template object and pass it to the constructor of the <code>DynamicParameters</code>. Here is an example that uses the <code>uspGetBillOfMaterials</code> stored procedure in AdventureWorks2022 database:

In [4]:
public class BillInput
{   
    public int StartProductID { get; set; } 
    public DateTime CheckDate { get; set; } 
}

public class Bill
{
    public int ProductAssemblyID { get; set; } 
    public string ComponentDesc { get; set; } 
    public int TotalQuantity { get; set; } 
}

using (var connection = new SqlConnection(connectionString))
{   
    string spName = "uspGetBillOfMaterials";
    BillInput template = new BillInput() {StartProductID = 3, CheckDate = new DateTime(2010, 5, 4)};
    DynamicParameters parameters = new DynamicParameters(template);   

    // Execute the stored procedure and map the results to a list of Employee objects
    List<Bill> bills = connection.Query<Bill>(spName, parameters, commandType: CommandType.StoredProcedure).ToList();

    // Print the results
    foreach (var bill in bills)
    {
        Console.WriteLine($@"{bill.ProductAssemblyID}, {bill.ComponentDesc}, {bill.TotalQuantity}");
    }
}

3, Cup-Shaped Race, 2
